In [ ]:
# Multiple processes need to start outside of notebook
from os.path import abspath, join
from subprocess import Popen, PIPE

script_path = abspath('')
script_exec_path = join(script_path,'compare-folders.py')
print("To execute:\n" + script_exec_path)

process = Popen(
    ['py', script_exec_path,
     '-c',
     'D:\\_Dimitrie' 
    ],
    stdout=PIPE, stderr=PIPE, 
    text=True  # Ensures strings instead of bytes
)

stdout, stderr = process.communicate()

print("Output:\n", stdout)
print("Error:\n", stderr)


In [ ]:
from ast import literal_eval
from csv import DictReader, field_size_limit
from os import listdir, remove
from os.path import isdir, splitdrive
from sys import maxsize

field_size_limit(maxsize)

keyword = "processed"
in_path = ""
for entry in listdir(script_path):
    if not isdir(entry):
        if ".csv" in entry:
            if keyword in entry:
                remove(entry)
            else:
                in_path = entry
        
print(in_path)

sep = '.'
p,ext = in_path.split(sep)
duplicates_path = p + "-" + keyword + sep + ext
singles_path = p + "-" + keyword + "-" + "single" + sep + ext
print(singles_path, duplicates_path)

with open(duplicates_path, mode = "w", newline='', encoding="utf-8") as d_file:
    with open(singles_path, mode = "w", newline='', encoding="utf-8") as s_file:
        with open(in_path, mode = "r", newline='', encoding="utf-8") as in_file:
            reader = DictReader(in_file)
            for i, row in enumerate(reader):
                value = literal_eval(row["paths"])
                if "desktop.ini" not in value[0]: # ignore system files
                    # write duplicate paths once:
                    s = set()
                    for entry in value:
                        entry = splitdrive(entry)[1]
                        s.add(entry)

                    if len(s):
                        if len(s) == 1:
                            file = s_file
                        if len(value) > 2: # duplicates
                            file = d_file

                    file.write(f"\t{i}\t")
                    for entry in s:
                        file.write(f"{entry} \r\n")
